In this notebook I have with the help of the article on QuantPy's website I implement a program that will find hte the optimal portfolio for a given collection of stocks at a given risk level. In this notebook I will use the past performances of the stocks in the portfolio to compute the returns to risk, however in practice this approach isn't necessarily the most appropriate. Things may change and there and the relationships between different stocks may change. However prediction of sharpe ratios, volatility and returns is a separate task. 

https://quantpy.com.au/python-for-finance/portfolio-optimisation/

In [85]:
# Some important packages that we will be using.

import pandas_datareader as pdr
import pandas as pd
import numpy as np 
import yfinance as yf 
import matplotlib as plt
import datetime as dt
import plotly.graph_objects as go
from datetime import date 
import scipy.optimize as sc
from scipy.optimize import Bounds, LinearConstraint, minimize
import bs4 as bs
import requests

In [86]:
# for fun you can take all of the snp 500 and and find optimal portfolios of them, warning this code takes a while to run if you want a quicker version use a shorter list of companies. 

resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})

tickers = []

for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    tickers.append(ticker)

# The free yfinance API is causing some issues with three tickers, so I have removed them.
    
tickers = [s.replace('\n', '') for s in tickers]
tickers.remove('BRK.B')
tickers.remove('BF.B')
tickers.remove('AME')

In [87]:
# First we need a function that will retrieve the data from Yfinance, we will do this using the pandas data reader. 
# Update the pandas datareader package has stopped working with Yahoo finance, this new code goes directly via yahoo finance. 

def getdata(stocks, datatype, startdate, enddate):
    yfdata = yfdata = yf.download(stocks, start = startdate, end= enddate)
    closedata = yfdata[datatype].copy()
    return closedata

In [88]:
# Next we need to take that data and work out the covariance between the stocks and work out the mean returns.
def meanrets(closedata):
    returns = closedata.pct_change()
    mean_returns = returns.mean()
    return mean_returns

def covar(closedata):
    returns = closedata.pct_change()
    covariance_matrix = returns.cov()
    return covariance_matrix
    

In [89]:
# Then we need to evaluate the different portfolios we have. we will do this using the returns and portfolio standard deviation.

def portfolio_returns (weights, mean_returns):
    # This is a yearly return not just the daily returns with 252 trading days in a year
    returns = np.sum(mean_returns*weights)*(years*252)
    return returns

def portfolio_standard (weights, covariance_matrix):
    # the standard deivation using the weights taking the dot product with the weights
    standard_deviaton = ((np.dot(weights.T,np.dot(covariance_matrix,weights)))**0.5)*((years*252)**0.5)
    return standard_deviaton

# The optimiser step

We want to minimize the variance, for a given level of return, we do this by using an optimiser and imposing restrictions. 
In this case our constrains are that the sum of hte weights are going to be 1, because we can only sum a maximum of 100% of our portfolio.
the second is that the returns need to be above a certain threshold, we will use different thresholds to see how low the variance can go at each.

The first step is working out what we want to minimize, in general though we want to maximise, in this case we want to maximize the sharpe, which is the relationship between returns- risk free rate of returns/ standard deviation. Our function however minimizes so we will just minimize the negative of the sharpe ratio. 

In [90]:
# The negative sharpe ratio function, that takes the weights, mean returns, and covariance matrix to output the negative of the sharpe ratio.

def negSharperatio(weights, mean_returns, covariance_matrix ,rrr):
    port_rets = portfolio_returns(weights,meanreturns)
    portfolio_stand = portfolio_standard(weights,covariance_matrix)
    return - (port_rets - rrr)/(portfolio_stand)

In [91]:
def maxsharpe(mean_returns, covariance_matrix, rrr, bound = (0,1)):
    # This will minimise the negative sharpe ratio to maximise the regular sharpe ratio.
    numberofassetts = len(mean_returns)
    args = (mean_returns,covariance_matrix, rrr)
    constraints = ({'type':'eq','fun': lambda x: np.sum(x)-1})
    bounds = tuple(bound for asset in range(numberofassetts))
    result = sc.minimize(negSharperatio, [1./numberofassetts]*numberofassetts,args=args, method= 'SLSQP', bounds=bounds, constraints=constraints )
    return result
    

In [92]:
# This function will work out the minimum variance that a portfolio can achieve. 
def minimizevariance(mean_returns, covariance_matrix, bound = (0,1)):
    numberofassetts = len(mean_returns)
    args = (covariance_matrix)
    constraints = ({'type':'eq','fun': lambda x: np.sum(x)-1})
    bounds = tuple(bound for asset in range(numberofassetts))
    result = sc.minimize(portfolio_standard,[1./numberofassetts]*numberofassetts,args=args ,method= 'SLSQP', bounds=bounds, constraints=constraints)
    return result

In [93]:
# Now we need a function that will find the minimum variance for any target returns. 
def efficentportfolio(mean_returns, covariance_matrix, target_returns, bound = (0,1)):
    numberofassetts = len(mean_returns)
    args = (covariance_matrix)
    constraints = ({'type':'eq','fun': lambda x: np.sum(x)-1},{'type':'eq', 'fun':lambda x: portfolio_returns(x,mean_returns)-target_returns})
    bounds = tuple(bound for asset in range(numberofassetts))
    efficentportfolio = sc.minimize( portfolio_standard , [1./numberofassetts]*numberofassetts ,args=args ,method= 'SLSQP', bounds=bounds, constraints=constraints)
    return efficentportfolio

In [94]:
# Lastly a function to call all of the above results, we will take the lowest volatility and the maximum sharpe ratio to use as the bounds for hte experiment.
# then calculate the 

def calculated_results(mean_returns, covariance_matrix, rrr, bound = (0,1)):
    # The maximum sharpe ratio allocaiton
    maxsr_portfolio = maxsharpe(mean_returns, covariance_matrix, rrr, bound = (0,1))
    maxsr_returns = portfolio_returns( maxsr_portfolio['x'],mean_returns)
    maxsr_std = portfolio_standard(maxsr_portfolio['x'],covariance_matrix)
    maxsr_allocation = pd.DataFrame(maxsr_portfolio['x'], index= mean_returns.index,columns=['allocation'])
    
    # The minimized volatility allocation
    minvol_portfolio = minimizevariance(mean_returns,covariance_matrix, bound = (0,1))
    minvol_returns = portfolio_returns( minvol_portfolio['x'],mean_returns)
    minvol_std = portfolio_standard( minvol_portfolio['x'],covariance_matrix)
    minvol_allocation = pd.DataFrame(minvol_portfolio['x'], index= mean_returns.index,columns=['allocation'])
    
    # Efficient frontier.
    efficentportfolios = []
    targetreturns = np.linspace(minvol_returns,maxsr_returns,20)
    for target in targetreturns:
        efficentportfolios.append(efficentportfolio(mean_returns,covariance_matrix,target,bound=(0,1))['fun'])
        
    return maxsr_returns, maxsr_std, maxsr_allocation, minvol_returns, minvol_std,minvol_allocation, efficentportfolios, targetreturns
    
    

In [95]:


def EF_graph(maxSR_returns, maxSR_std, maxSR_allocation, minVol_returns, minVol_std, minVol_allocation, efficientList, targetReturns):
    # Plotting the maximum sharpe ratio portfolio.
    MaxSharpeRatio = go.Scatter(
        name='Maximium Sharpe Ratio',
        mode='markers',
        x=[round(maxSR_std*100,2)],
        y=[round(maxSR_returns*100,2)],
        marker=dict(color='red',size=14,line=dict(width=3, color='black'))
    )
    # Plotting the minimum volatility portfolio.
    MinVol = go.Scatter(
        name='Mininium Volatility',
        mode='markers',
        x=[round(minVol_std*100,2)],
        y=[round(minVol_returns*100,2)],
        marker=dict(color='green',size=14,line=dict(width=3, color='black'))
    )
    
    # The frontier made up of the lowest volatility possible at each level of returns between the returns of the lowest volatility porfolio and the maximum sharpe ratio portfolio.\
        
    #Efficient Frontier
    EF_curve = go.Scatter(
        name='Efficient Frontier',
        mode='lines',
        x=[round(ef_std*100, 2) for ef_std in efficientList],
        y=[round(target*100, 2) for target in targetReturns],
        line=dict(color='black', width=4, dash='dashdot')
    )
    data = [MaxSharpeRatio, MinVol, EF_curve]
    layout = go.Layout(
        title = 'Portfolio Optimisation with the Efficient Frontier',
        yaxis = dict(title='Annualised Return (%)'),
        xaxis = dict(title='Annualised Volatility (%)'),
        showlegend = True,
        legend = dict(
            x = 0.75, y = 0, traceorder='normal',
            bgcolor='#E2E2E2',
            bordercolor='black',
            borderwidth=2),
        width=1000,
        height=1000)
    
    fig = go.Figure(data=data, layout=layout)
    return fig.show()

In [96]:
# the number of years worth of data to calculate over
years = 1
# the end date of the trial, by default set to todays date
enddate = dt.datetime.now() 
# the start date of the trial 
startdate = enddate - dt.timedelta(days = 365*years)
# The stocks to be included in the portfolio, leave as tickers for the whole snp500, or add your own list. 
# The code is set up for a 252 trading days year, it will need to be changed if other assets are included. 
stocks = tickers
# Using the adjusted close to take dividence and splits into account.
datatype = 'Adj Close'

data = getdata(stocks,datatype,startdate, enddate)

meanreturns = meanrets(data)
covariance = covar(data)

maxsr_returns, maxsr_std, maxsr_allocation, minvol_returns, minvol_std,minvol_allocation, efficentportfolios, targetreturns = calculated_results(meanreturns, covariance, rrr=0.03, bound = (0,1))

EF_graph(maxsr_returns, maxsr_std, maxsr_allocation, minvol_returns, minvol_std,minvol_allocation, efficentportfolios, targetreturns)


[*********************100%***********************]  500 of 500 completed
